# Data collection

In [1]:
import os
import time
import uuid
import cv2
import tensorflow as tf
import numpy as np

In [5]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [9]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Data labeling

In [10]:
!pip install labelme

  Using cached labelme-5.0.1.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.7/7.7 MB 2.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 9.2/9.2 MB 2.4 MB/s eta 0:00:00
  Created wheel for labelme: filename=labelme-5.0.1-py3-none-any.whl size=1466188 sha256=7a7d3a60b2a04e17db8d2891a65105d069a04f871025b30eda6f1b40759a5072
  Stored in directory: c:\users\honda\appdata\local\pip\cache\wheels\29\9b\b4\41b9f5488d5859bef0cf383385f88ac4bcc521b19b47a71a4c
  Created wheel for imgviz: filename=imgviz-1.5.0-py3-none-any.whl size=7680439 sha2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mizani 0.7.4 requires matplotlib>=3.5.0, but you have matplotlib 3.2.2 which is incompatible.
mediapipe 0.8.9.1 requires attrs>=19.1.0, but you have attrs 18.1.0 which is incompatible.


In [11]:
!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: C:\Users\Honda\.labelmerc


# Real time prediction

In [3]:
from tensorflow.keras.models import load_model

In [4]:
facetracker=load_model('facetracker.h5')

In [10]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'Honda', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
   

cap.release()
cv2.destroyAllWindows()